In [ ]:
! pip install -r requirements.txt

In [15]:
import json
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    TextGenerationPipeline
)
from huggingface_hub import login
import gradio as gr

### Account config

In [16]:
config_data = json.load(open("config.json"))
HF_token = config_data["HF_token"]

In [42]:

login(token=HF_token) # Replace YOUR_HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

### Quantisation config

In [19]:
bnb_conifig = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)                                               #quantisation

## loading tkenizer and LLM

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token= HF_token)

tokenizer.pad_token = tokenizer.eos_token       #end of sequence | identify the end of S


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [21]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             token= HF_token,
                                             quantization_config=bnb_conifig,
                                             device_map="auto")

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [22]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.5,
    top_p=0.95,
    repetition_penalty=1.15,
    return_full_text=True,
)

In [29]:
prompt = "What is the machine learning?"

In [30]:
def get_response(prompt):
    sequence = text_generator(prompt)
    gen_text = sequence[0]["generated_text"]
    return gen_text

In [ ]:
response = get_response(prompt)
print(response[len(prompt):])

In [40]:
def generate_response(prompt):
  sequence = text_generator(prompt)
  gen_text = sequence[0]["generated_text"]
  return gen_text[len(prompt):]

demo = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs="text",
    title="Llama 3B Instruct",
    description="Ask me anything!",
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6b3350b5f596dbd6f3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
